In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [9]:
#Load the spam dataset:

data = np.array(pd.read_csv('../data/spambase.data', header=None))

X = data[:,:-1] # features
y = data[:,-1] # Last column is label
#  zeros labels must be negative (-1) for AdaBoost
y[y == 0] = -1 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True, stratify=y)



Excercise 1. AdaBoot 
Implement AdaBoost using Python (incl. Numpy etc.) and use it on the SPAM-Dataset

1.The weak classifiers should be decision stumps (i.e. decision trees with one node).

(a) Print a confusion matrix.

(b) Is AdaBoost better when using stronger weak learners? Why or why not? Compare your results to using depth-2 decision trees.


In [10]:
# A decision stump is a machine learning model consisting of a one-level decision tree.
# That is, it is a decision tree with one internal node (the root) 
# which is immediately connected to the terminal nodes (its leaves).

# For continuous features, usually, some threshold feature value is selected, 
# and the stump contains two leaves — for values below and above the threshold. 
class OneLevelDecisionTree():
    def fit(self, feature_column, neg_label, pos_label, feature):
        self.feature = feature_column
        self.neg_label = neg_label
        self.pos_label = pos_label
        self.threshold = feature
        
    def predict(self, X):
        predictions = []
        
        if self.feature >= self.threshold:
            return self.neg_label
        else:
            return self.pos_label
            
    
def classifier_boost(X, cls):
    boost = []
    neg_label = -1
    pos_label = 1
    
    for column in range(X.shape[1]):
        # passing by columns
        # get every unique feature in ordered way
        feature_set = sorted(set(X[:, column]))
        
        # threshold feature value is selected for every feature
        for threshold in feature_set:
            boost.append(cls.fit(column, neg_label, pos_label, threshold))
            
    return boost

In [11]:
class AdaBoost():
    def __init__(self, cls_boost, boost_size):
        self.cls_boost = cls_boost
        self.boost_size = boost_size       
        self.classifiers = []
        self.weights = []
        self.error = []
        
    # from the lecture:
    # error calculation looks how many times the prediction
    # of the model was wrong
    def compute_error(self, preds):
        for i in range(len(preds)):
            if preds[i] != self.y[i]:
                self.error.append(self.weights[i])
                
        
    def fit_train(self, X, y):
        self.X = X
        self.y = y
        
        predictions = []
        
        # Step 1
        # initialise w_i = 1/N for all i from [1..N]
        N = len(y) + 1
        for i in range(1, N):
            w = 1/N
            self.weights.append(w)
        
        # Step 2.a - 1
        # for m from [1..M] of boost size
        # train a classifier f_m(x) -> [-1, 1] on X
        # --> DONE with classifier_boost() and saved as self.cls_boost
        
        # Step 2.a - 2
        # here is probably supposed to be a prediction step...
        for cls in self.cls_boost:
            predictions.append(cls.predict(X))
        
        # Step 2.b
        # Compute classification error    
        for m in range(1, self.boost_size + 1):
            self.compute_error(predictions)
            # from lecture: sum of all the weights that were missclassified 
            # devided by the sum of all weights
            E_m = np.sum(self.error) / np.sum(self.weights)
            
            # Step 2.c
            # Compute classifier weight
            
            alpha_m = np.log((1 - E_m) / E_m)
            
    
    def fit(self, X, y):
        scouting_matrix = np.array([clf.predict(X) != y for clf in self.classifier_pool])
        w = np.ones(len(y)) # Initially, all weights are the same
        
        for _ in range(self.num_classifiers):
            # Step 1
            errors = scouting_matrix@w
            best_remaining = errors.argmin()
            
            # Step 2
            We = errors[best_remaining]
            W = w.sum()
            em = (W - We) / W
                        
            self.classifiers += [self.classifier_pool[best_remaining]]
            self.weights += [0.5 * -np.log((1 - em) / em)] # alphas
            
            # Step 3
            w = w * np.exp(np.where(scouting_matrix[best_remaining], 1, -1) * self.weights[-1])
            scouting_matrix = np.delete(scouting_matrix, best_remaining, axis=0)
            del self.classifier_pool[best_remaining]
        
    def predict(self, X):
        preds = np.array([cl.predict(X) for cl in self.classifiers])
        weighted_preds = np.dot(self.weights, preds)
        return np.where(weighted_preds >= 0, 1, -1)

        

In [18]:
one_cls = OneLevelDecisionTree()
boost = classifier_boost(X_train, one_cls)
print ("Weak: {} of type {}".format(len(boost), type(one_cls).__name__))

Weak: 13293 of type OneLevelDecisionTree


In [ ]:
boost_size = 50
ada = AdaBoost(boost, boost_size)
ada.fit_train(X_train, y_train)
ada_predictions = model.predict(X_test)

In [21]:
def calculate_accuracy(labels, predictions):
        return np.mean(labels == predictions)


In [22]:
print(calculate_accuracy(y_train, ada_predictions))

NameError: name 'ada_predictions' is not defined

In [19]:

# So I will use my code from previous assignment to build a one
# THIS CELL IS COPIED FROM THE PREVIOUS ASSIGNMENT AND CONTAINS IMPLEMENTATION FOR A DECISSION TREE 


def cross_entropy(p):
        if p == 1 or p == 0: 
            # The entropy is zero if one event is certain
            return 0
        return - (p * np.log(p) + (1-p) * np.log((1-p)))

# Weight of a child node is number of samples in the node/total samples of all child nodes. 
# Similarly information gain is calculated with gini score. 
def children_entropy(feature, y):
    right = (feature == True).sum()/len(feature)
    left = 1 - right
    
    p = np.sum(y[feature])/len(y[feature]) 
    q = np.sum(y[np.invert(feature)])/len(y[np.invert(feature)])
    
    entropy_right = right * cross_entropy(p)
    entropy_left = left * cross_entropy(q)
    total_entropy = entropy_right + entropy_left
    return total_entropy, q, p

#====================================

class DecisionTree():
    
    def __init__(self, height=7):
        self.min_size = 4
        self.height = height
    
    # fit a basic binary tree for 2 classes classificaton 
    def fit(self, X, y):
        self.tree_size = 2**self.height - 1
        #print(self.tree_size)
        self.tmp_size = 2**(self.height + 1) - 1
        self.features = X.shape[1]
        self.tree = np.full(self.tmp_size, -1)
        self.tree_tmp = np.full(self.tmp_size + 1, -1)
        self.split_tree(X, y, 0)
    
    # binary tree
    def left_tree(self, leaf):
        return 2 * leaf + 1
    
    def right_tree(self, leaf):
        return 2 * leaf + 2
    
    
    def predict(self, X):
        predictions = []
        for x in X:
            idx = 0
            leaf = self.tree[idx]
            while self.tree[self.left_tree(idx)] != -1 or self.tree[self.right_tree(idx)] != -1:
                #print("idx:", idx)
                #print("leaf:", idx)

                if leaf >= self.tree_size:
                    return
                
                if x[leaf]:
                    idx = self.right_tree(idx)
                    #print("--------> right")
                else:
                    idx = self.left_tree(idx)
                    #print("left <--------")
                prediction = self.tree_tmp[idx]
                leaf = self.tree[idx]
            predictions += [prediction]
        return predictions
    
    
    def split_data(self, index, value, X):
        left, right = list(), list()
        for row in X:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right
        
    
    def split_tree(self, X, y, leaf):
  
        # parent node is a leaf
        #print("leaf", leaf)
        if leaf >= self.tree_size:
            return
        
        entropies = np.full(self.features, np.inf) 
        left = np.empty(self.features)
        right = np.empty(self.features)
        
        # for every feature variable
        for i, feature in enumerate(X.T):
            if np.sum(feature) == 0 or np.sum(np.invert(feature)) == 0:
                continue 
            entropies[i], left[i], right[i] = children_entropy(feature, y)
        
        min_entropy = np.argmin(entropies)
        
        right = X[:,min_entropy]
        left = np.invert(right)
        #print(left)
        
        #print("min_entropy", min_entropy)
        self.tree[leaf] = min_entropy
        if min_entropy < len(self.tree_tmp):
            if (min_entropy < len(left)) and (min_entropy < len(right)):
                self.tree_tmp[self.left_tree(leaf)] = left[min_entropy]
                self.tree_tmp[self.right_tree(leaf)] = right[min_entropy]
        
        if len(y[right]) == 0 or len(y[left]) == 0:
            return
        # grow tree 
        if leaf >= self.min_size:
            return
        self.split_tree(X[left], y[left], self.left_tree(leaf))
        self.split_tree(X[right], y[right], self.right_tree(leaf))
            



Excercise 2 (Bonus).

Viola-Jones Face Detection
Implement the Viola-Jones algorithm (without the cascade mechanism) and use it on a LFW-Face-subsetto classify faces.

(a) Visualize the top ten face classifiers

Excercise 3 (Bonus).
Cascade-Classification

Implement a cascade algorithm to classify faces in a picture of your choice 
(there should be more than a face on your image, e.g. skimage.data.astronaut())